# SITCOM-771
Craig Lage - 12-May-23 \

Here is what was requested:
* How did the force profile change over time?
* Find all bump test patterns in the EFD for a given FA.
* Select nominal force patterns and average some of them.
* Overplot Nomial force pattern to actual force pattern.
* Do the subtraction.
* Plot the residuals.

This notebook does those things


## Prepare the notebook

In [ ]:
# Directory to store the data
dataDir = "/home/c/cslage/u/MTM1M3/data/"

# Times of single bump test
singleStart = "2023-04-17T10:00:00"
singleEnd = "2023-04-17T11:15:00"

# Times of multiple bump tests
multipleStart = "2020-06-01T00:00:00"
multipleEnd = "2023-04-20T00:00:00"

# Times of test with bump test residuals
residualStart = "2023-04-28T18:10:00"
residualEnd = "2023-04-28T19:18:00"

In [ ]:
import sys, time, os, asyncio, glob
from datetime import datetime
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
import pickle as pkl
from astropy.time import Time, TimeDelta
import lsst.ts.cRIOpy.M1M3FATable as M1M3FATable
from scipy.interpolate import UnivariateSpline
from lsst_efd_client import EfdClient

In [ ]:
client = EfdClient('usdf_efd')
FATABLE = M1M3FATable.FATABLE

## Given an actuator ID, this plots the bump test result for a single test

In [ ]:
async def plotBumpTestResults(fig, bumps, id):
    thisBump = bumps[bumps['actuatorId']==id]
    index = M1M3FATable.actuatorIDToIndex(id)
    # The pass/fail results are actually in the next test.
    lastThisBumpIndex = bumps[bumps['actuatorId']==id].last_valid_index()
    passFail = bumps.iloc[bumps.index.get_loc(lastThisBumpIndex)+1]
    primaryBump = f"primaryTest{FATABLE[index][M1M3FATable.FATABLE_ZINDEX]}"
    primaryForce = f"zForce{FATABLE[index][M1M3FATable.FATABLE_ZINDEX]}"
    if FATABLE[index][M1M3FATable.FATABLE_TYPE] == 'DAA':
        if FATABLE[index][M1M3FATable.FATABLE_ORIENTATION] in ['+Y', '-Y']:
            secondaryBump = f"secondaryTest{FATABLE[index][M1M3FATable.FATABLE_SINDEX]}"
            secondaryForce = f"yForce{FATABLE[index][M1M3FATable.FATABLE_YINDEX]}"
            secondaryName = FATABLE[index][M1M3FATable.FATABLE_ORIENTATION]
        else:
            secondaryBump = f"secondaryTest{FATABLE[index][M1M3FATable.FATABLE_SINDEX]}"
            secondaryForce = f"xForce{FATABLE[index][M1M3FATable.FATABLE_XINDEX]}"
            secondaryName = FATABLE[index][M1M3FATable.FATABLE_ORIENTATION]
    else:
        secondaryName = None

    plt.subplots_adjust(wspace=0.3)
    plt.subplot(1,2,1)
    plotStart = thisBump[thisBump[primaryBump]==2]['timestamp'].values[0] - 1.0
    plotEnd = plotStart + 14.0 #thisBump[thisBump[primaryBump]==5]['timestamp'].values[0] + 2.0
    start = Time(plotStart, format='unix_tai', scale='tai')
    end = Time(plotEnd, format='unix_tai', scale='tai')
    forces = await client.select_time_series("lsst.sal.MTM1M3.forceActuatorData", [primaryForce, 'timestamp'], start.utc, end.utc)
    times = forces['timestamp'].values
    t0 = times[0]
    times -= t0
    plotStart -= t0
    plotEnd -= t0
    plt.title(f"Primary - Z - ID:{id}")
    plt.plot(times, forces[primaryForce].values)
    if passFail[primaryBump] == 6:
        plt.text(2.0, 350.0, "PASSED", color='g')
    elif passFail[primaryBump] == 7:
        plt.text(2.0, 350.0, "FAILED", color='r')

    plt.xlim(plotStart, plotEnd)
    plt.ylim(-400,400)
    plt.xlabel("Time (seconds)")
    plt.ylabel("Force (nt)")
    plt.subplot(1,2,2)
    if secondaryName is not None:
        plt.title(f"Secondary - {secondaryName} - ID:{id}")
        plotStart = thisBump[thisBump[secondaryBump]==2]['timestamp'].values[0] - 1.0
        plotEnd = plotStart + 14.0 #thisBump[thisBump[secondaryBump]==5]['timestamp'].values[0] + 2.0
        start = Time(plotStart, format='unix_tai', scale='tai')
        end = Time(plotEnd, format='unix_tai', scale='tai')
        forces = await client.select_time_series("lsst.sal.MTM1M3.forceActuatorData", [secondaryForce, 'timestamp'], start.utc, end.utc)
        times = forces['timestamp'].values
        t0 = times[0]
        times -= t0
        plotStart -= t0
        plotEnd -= t0
        plt.plot(times, forces[secondaryForce].values)
        if passFail[secondaryBump] == 6:
            plt.text(2.0, 350.0, "PASSED", color='g')
        elif passFail[secondaryBump] == 7:
            plt.text(2.0, 350.0, "FAILED", color='r')
        plt.xlim(plotStart, plotEnd)
        plt.ylim(-400,400)
        plt.xlabel("Time (seconds)")
        plt.ylabel("Force (nt)")
    else:
        plt.title("No Secondary")
        plt.xticks([])
        plt.yticks([])
    return

## Now make the plot for a single actuator

In [ ]:
bumps = await client.select_time_series("lsst.sal.MTM1M3.logevent_forceActuatorBumpTestStatus", "*", \
                                        Time(singleStart, scale='utc'), Time(singleEnd, scale='utc'))
timestamp = bumps.index[0].isoformat().split('.')[0].replace('-','').replace(':','')
fig = plt.figure(figsize=(10,5))
id = 227
await plotBumpTestResults(fig, bumps, id)
timestamp = bumps.index[0].isoformat().split('.')[0].replace('-','').replace(':','')
plt.savefig(dataDir+f"Bump_Test_{id}_{timestamp}.png")

## This will make a plot of all of the actuators in a given bump test

In [ ]:
bumps = await client.select_time_series("lsst.sal.MTM1M3.logevent_forceActuatorBumpTestStatus", "*", \
                                        Time(singleStart, scale='utc'), Time(singleEnd, scale='utc'))
timestamp = bumps.index[0].isoformat().split('.')[0].replace('-','').replace(':','')
pdf = PdfPages(dataDir + f"Bump_Tests_{timestamp}.pdf")

for index in range(len(FATABLE)):
    id = FATABLE[index][M1M3FATable.FATABLE_ID]
    fig = plt.figure(figsize=(10,5))
    await plotBumpTestResults(fig, bumps, id)
    pdf.savefig(fig)  # saves the current figure into a pdf page
    plt.close()
pdf.close()

## Now let's look at the history of many bump tests

In [ ]:
async def plotMultipleBumpTestResults(fig, manyBumps, id):
    theseBumps = manyBumps[manyBumps['actuatorId']==id]
    index = M1M3FATable.actuatorIDToIndex(id)
    primaryBump = f"primaryTest{FATABLE[index][M1M3FATable.FATABLE_ZINDEX]}"
    primaryForce = f"zForce{FATABLE[index][M1M3FATable.FATABLE_ZINDEX]}"
    if FATABLE[index][M1M3FATable.FATABLE_TYPE] == 'DAA':
        if FATABLE[index][M1M3FATable.FATABLE_ORIENTATION] in ['+Y', '-Y']:
            secondaryBump = f"secondaryTest{FATABLE[index][M1M3FATable.FATABLE_SINDEX]}"
            secondaryForce = f"yForce{FATABLE[index][M1M3FATable.FATABLE_YINDEX]}"
            secondaryName = FATABLE[index][M1M3FATable.FATABLE_ORIENTATION]
        else:
            secondaryBump = f"secondaryTest{FATABLE[index][M1M3FATable.FATABLE_SINDEX]}"
            secondaryForce = f"xForce{FATABLE[index][M1M3FATable.FATABLE_XINDEX]}"
            secondaryName = FATABLE[index][M1M3FATable.FATABLE_ORIENTATION]
    else:
        secondaryName = None
    plt.subplots_adjust(wspace=0.3)
    plt.suptitle(f"Multiple bump tests Actuator ID {id}", fontsize=18)

    # Now find the separate tests
    times = theseBumps['timestamp'].values
    startTimes = []
    endTimes = []
    for i, time in enumerate(times):
        if i == 0:
            startTimes.append(time)
            continue
        if (time - times[i-1]) > 60.0:
            startTimes.append(time)
            endTimes.append(times[i-1])
    endTimes.append(times[-1])
    numPlots = 0
    passCount = 0
    failCount = 0
    for i in range(len(startTimes)):
        startTime = startTimes[i]
        endTime = endTimes[i]
        thisBump = theseBumps[(theseBumps['timestamp'] >= startTime) & (theseBumps['timestamp'] <= endTime)]
        try:
            numPlots += 1
            plt.subplot(1,2,1)
            plotStart = thisBump[thisBump[primaryBump]==2]['timestamp'].values[0] - 1.0
            plotEnd = plotStart + 14.0
            start = Time(plotStart, format='unix_tai', scale='tai')
            end = Time(plotEnd, format='unix_tai', scale='tai')
            forces = await client.select_time_series("lsst.sal.MTM1M3.forceActuatorData", [primaryForce, 'timestamp'], \
                                                     start.utc, end.utc)
            times = forces['timestamp'].values
            t0 = times[0]
            times -= t0
            plotStart -= t0
            plotEnd -= t0
            lastIndex = bumps[bumps['actuatorId']==id].last_valid_index()
            passFail = bumps.iloc[bumps.index.get_loc(lastIndex)+1][primaryBump]
            #print(i, passFail)
            if passFail == 6:
                passCount += 1
                #print("PASSED")
            elif passFail == 7:
                failCount += 1
                #print("FAILED")

            plt.title("Primary   Z")
            plt.plot(times, forces[primaryForce].values)
            plt.xlim(plotStart, plotEnd)
            plt.ylim(-400,400)
            plt.xlabel("Time (seconds)")
            plt.ylabel("Force (nt)")
            plt.subplot(1,2,2)
            if secondaryName is not None:
                plt.title(f"Secondary   {secondaryName}")
                plotStart = thisBump[thisBump[secondaryBump]==2]['timestamp'].values[0] - 1.0
                plotEnd = plotStart + 14.0
                start = Time(plotStart, format='unix_tai', scale='tai')
                end = Time(plotEnd, format='unix_tai', scale='tai')
                forces = await client.select_time_series("lsst.sal.MTM1M3.forceActuatorData", [secondaryForce, 'timestamp'], \
                                                         start.utc, end.utc)
                times = forces['timestamp'].values
                t0 = times[0]
                times -= t0
                plotStart -= t0
                plotEnd -= t0
                plt.plot(times, forces[secondaryForce].values)
                plt.xlim(plotStart, plotEnd)
                plt.ylim(-400,400)
                plt.xlabel("Time (seconds)")
                plt.ylabel("Force (nt)")
            else:
                plt.title("No Secondary")
                plt.xticks([])
                plt.yticks([])
        except:
            continue
            
    plt.subplot(1,2,1)
    plt.text(2.0, 350, f"{numPlots} tests, {passCount} passed, {failCount} failed")
    plt.subplot(1,2,2)
    plt.text(2.0, 350, f"{numPlots} tests, {passCount} passed, {failCount} failed")
    return

## Now make the history plot for a single actuator

In [ ]:
manyBumps = await client.select_time_series("lsst.sal.MTM1M3.logevent_forceActuatorBumpTestStatus", "*", \
                                            Time(multipleStart, scale='utc'), Time(multipleEnd, scale='utc'))
id = 227
fig = plt.figure(figsize=(10,5))
await plotMultipleBumpTestResults(fig, manyBumps, id)
plt.savefig(dataDir + f"Bump_Test_{id}_Multiple.png")

## This will make a plot of the histories of all of the actuators

In [ ]:
manyBumps = await client.select_time_series("lsst.sal.MTM1M3.logevent_forceActuatorBumpTestStatus", "*", \
                                            Time(multipleStart, scale='utc'), Time(multipleEnd, scale='utc'))
pdf = PdfPages(dataDir + f"Bump_Histories.pdf")

for index in range(len(FATABLE)):
    id = FATABLE[index][M1M3FATable.FATABLE_ID]
    fig = plt.figure(figsize=(10,5))
    await plotBumpTestResults(fig, bumps, id)
    pdf.savefig(fig)  # saves the current figure into a pdf page
    plt.close()
pdf.close()

## Now generate the dictionary of average splines
## This will be used to plot the residuals
## Note that this only needs to be done once.  After it is run, the average splines are pickled and can be retrieved for future tests.

In [ ]:
async def generateAverageBumpTest(manyBumps, averageSplineDict, id):
    # Get representative splines to be used to weed out bad runs
    [averagePrimarySpline, averageSecondarySpline] = averageSplineDict[999]
    theseBumps = manyBumps[manyBumps['actuatorId']==id]
    index = M1M3FATable.actuatorIDToIndex(id)
    primaryBump = f"primaryTest{FATABLE[index][M1M3FATable.FATABLE_ZINDEX]}"
    primaryForce = f"zForce{FATABLE[index][M1M3FATable.FATABLE_ZINDEX]}"
    if FATABLE[index][M1M3FATable.FATABLE_TYPE] == 'DAA':
        if FATABLE[index][M1M3FATable.FATABLE_ORIENTATION] in ['+Y', '-Y']:
            secondaryBump = f"secondaryTest{FATABLE[index][M1M3FATable.FATABLE_SINDEX]}"
            secondaryForce = f"yForce{FATABLE[index][M1M3FATable.FATABLE_YINDEX]}"
            secondaryName = FATABLE[index][M1M3FATable.FATABLE_ORIENTATION]
        else:
            secondaryBump = f"secondaryTest{FATABLE[index][M1M3FATable.FATABLE_SINDEX]}"
            secondaryForce = f"xForce{FATABLE[index][M1M3FATable.FATABLE_XINDEX]}"
            secondaryName = FATABLE[index][M1M3FATable.FATABLE_ORIENTATION]
    else:
        secondaryName = None

    # Now find the separate tests
    times = theseBumps['timestamp'].values
    startTimes = []
    endTimes = []
    for i, time in enumerate(times):
        if i == 0:
            startTimes.append(time)
            continue
        if (time - times[i-1]) > 60.0:
            startTimes.append(time)
            endTimes.append(times[i-1])
    endTimes.append(times[-1])
    numPlots = 0
    primarySplines = []
    secondarySplines = []
    for i in range(len(startTimes)):
        startTime = startTimes[i]
        endTime = endTimes[i]
        thisBump = theseBumps[(theseBumps['timestamp'] >= startTime) & (theseBumps['timestamp'] <= endTime)]
        # The pass/fail results are actually in the next test.
        lastThisBumpIndex = bumps[bumps['actuatorId']==id].last_valid_index()
        passFail = bumps.iloc[bumps.index.get_loc(lastThisBumpIndex)+1]
        if passFail[primaryBump] == 7:
            # Don't include fails
            continue
        try:
            plotStart = thisBump[thisBump[primaryBump]==2]['timestamp'].values[0] - 1.0
            plotEnd = plotStart + 14.0
            start = Time(plotStart, format='unix_tai', scale='tai')
            end = Time(plotEnd, format='unix_tai', scale='tai')
            forces = await client.select_time_series("lsst.sal.MTM1M3.forceActuatorData", [primaryForce, 'timestamp'], start.utc, end.utc)
            times = forces['timestamp'].values
            t0 = times[0]
            times -= t0
            primaryForces = forces[primaryForce].values
            if averagePrimarySpline is not None:
                rmsError = np.sqrt(np.mean((primaryForces-averagePrimarySpline(times))**2))
            else:
                rmsError = 0.0
            if rmsError < 50.0:
                # Only include good fits in the average
                primarySpline = UnivariateSpline(times, primaryForces, s=0.0)
                primarySplines.append(primarySpline)
            if secondaryName is not None:
                if passFail[secondaryBump] == 7:
                    # Don't include fails
                    continue
                plotStart = thisBump[thisBump[secondaryBump]==2]['timestamp'].values[0] - 1.0
                plotEnd = plotStart + 14.0
                start = Time(plotStart, format='unix_tai', scale='tai')
                end = Time(plotEnd, format='unix_tai', scale='tai')
                forces = await client.select_time_series("lsst.sal.MTM1M3.forceActuatorData", [secondaryForce, 'timestamp'], start.utc, end.utc)
                times = forces['timestamp'].values
                t0 = times[0]
                times -= t0
                secondaryForces = forces[secondaryForce].values
                if averageSecondarySpline is not None:
                    rmsError = np.sqrt(np.mean((secondaryForces-averageSecondarySpline(times))**2))
                else:
                    rmsError = 0.0
                if rmsError < 50.0:
                    # Only include good fits in the average
                    secondarySpline = UnivariateSpline(times, secondaryForces, s=0.0)
                    secondarySplines.append(secondarySpline)
        except:
            continue
    # Now calculate the average spline
    ts = np.linspace(0,14,5000)
    fs = np.zeros_like(ts)
    numSplines = 0
    for spline in primarySplines:
        numSplines += 1
        fs += spline(ts)
    fs /= numSplines
    averagePrimarySpline = UnivariateSpline(ts, fs)
    fs = np.zeros_like(ts)
    if secondaryName is not None:
        numSplines = 0
        for spline in secondarySplines:
            numSplines += 1
            fs += spline(ts)
        fs /= numSplines
        averageSecondarySpline = UnivariateSpline(ts, fs)
    else:
        averageSecondarySpline = None
    return [averagePrimarySpline, averageSecondarySpline]

## This cell generates the dictionary with a set of plots of the average spline

In [ ]:
# First seed the averageSplineDict with a typical bump test
# This is used to weed out bad tests
averageSplineDict = {}
averageSplineDict[999] = [None, None]
[averagePrimarySpline, averageSecondarySpline] = await generateAverageBumpTest(manyBumps, averageSplineDict, 227)
averageSplineDict[999] = [averagePrimarySpline, averageSecondarySpline]
# Now run all of the actuators
pdf = PdfPages(dataDir + "Average_Spline_Dict.pdf")
for index in range(len(FATABLE)):
    id = FATABLE[index][M1M3FATable.FATABLE_ID]
    primaryName = 'Z'
    if FATABLE[index][M1M3FATable.FATABLE_TYPE] == 'DAA':
        if FATABLE[index][M1M3FATable.FATABLE_ORIENTATION] in ['+Y', '-Y']:
            secondaryName = FATABLE[index][M1M3FATable.FATABLE_ORIENTATION]
        else:
            secondaryName = FATABLE[index][M1M3FATable.FATABLE_ORIENTATION]
    else:
        secondaryName = None

    [averagePrimarySpline, averageSecondarySpline] = await generateAverageBumpTest(manyBumps, averageSplineDict, id)
    averageSplineDict[id] = [averagePrimarySpline, averageSecondarySpline]
    fig = plt.figure(figsize=(10,5))
    ts = np.linspace(0,14,5000)
    plt.suptitle(f"Average Spline bumps for ID = {id}")
    plt.subplot(1,2,1)
    plt.title(f"Primary  {primaryName}")
    plt.plot(ts, averagePrimarySpline(ts))
    plt.subplot(1,2,2)
    plt.title(f"Secondary   {secondaryName}")
    if averageSecondarySpline is not None:
        plt.plot(ts, averageSecondarySpline(ts))
    else:
        plt.xticks([])
        plt.yticks([])
                  
    pdf.savefig(fig)  # saves the current figure into a pdf page
    plt.clf()
pdf.close()


## Pickle the dictionary for future use

In [ ]:
filename = dataDir + 'average_spline_dict.pkl'
file = open(filename, 'wb')
pkl.dump(averageSplineDict, file)
file.close()

In [ ]:
filename = dataDir + 'average_spline_dict.pkl'
file = open(filename, 'rb')
averageSplineDict = pkl.load(file)
file.close()

## Now plot the residuals against the average

In [ ]:
async def plotBumpResultsAndResiduals(fig, bumps, averageSplineDict, id):
    [averagePrimarySpline, averageSecondarySpline] = averageSplineDict[id]
    thisBump = bumps[bumps['actuatorId']==id]
    index = M1M3FATable.actuatorIDToIndex(id)
    # The pass/fail results are actually in the next test.
    lastThisBumpIndex = bumps[bumps['actuatorId']==id].last_valid_index()
    passFail = bumps.iloc[bumps.index.get_loc(lastThisBumpIndex)+1]
    primaryBump = f"primaryTest{FATABLE[index][M1M3FATable.FATABLE_ZINDEX]}"
    primaryForce = f"zForce{FATABLE[index][M1M3FATable.FATABLE_ZINDEX]}"
    if FATABLE[index][M1M3FATable.FATABLE_TYPE] == 'DAA':
        if FATABLE[index][M1M3FATable.FATABLE_ORIENTATION] in ['+Y', '-Y']:
            secondaryBump = f"secondaryTest{FATABLE[index][M1M3FATable.FATABLE_SINDEX]}"
            secondaryForce = f"yForce{FATABLE[index][M1M3FATable.FATABLE_YINDEX]}"
            secondaryName = FATABLE[index][M1M3FATable.FATABLE_ORIENTATION]
        else:
            secondaryBump = f"secondaryTest{FATABLE[index][M1M3FATable.FATABLE_SINDEX]}"
            secondaryForce = f"xForce{FATABLE[index][M1M3FATable.FATABLE_XINDEX]}"
            secondaryName = FATABLE[index][M1M3FATable.FATABLE_ORIENTATION]
    else:
        secondaryName = None

    plt.subplots_adjust(wspace=0.3)
    plt.suptitle(f"Bump Test with Residuals. Actuator ID {id}", fontsize=18)
    plotStart = thisBump[thisBump[primaryBump]==2]['timestamp'].values[0] - 1.0
    plotEnd = plotStart + 14.0 
    start = Time(plotStart, format='unix_tai', scale='tai')
    end = Time(plotEnd, format='unix_tai', scale='tai')
    forces = await client.select_time_series("lsst.sal.MTM1M3.forceActuatorData", [primaryForce, 'timestamp'], start.utc, end.utc)
    times = forces['timestamp'].values
    t0 = times[0]
    times -= t0
    primaryForces = forces[primaryForce].values
    residuals = primaryForces-averagePrimarySpline(times)
    rmsError = np.sqrt(np.mean(residuals**2))
    plotStart -= t0
    plotEnd -= t0
    plt.subplot(2,2,1)
    plt.title("Primary - Z")
    plt.plot(times, averagePrimarySpline(times), label='Average')
    plt.plot(times, primaryForces, label='Data')
    if passFail[primaryBump] == 6:
        plt.text(2.0, 350.0, "PASSED", color='g')
    elif passFail[primaryBump] == 7:
        plt.text(2.0, 350.0, "FAILED", color='r')
    plt.xlim(plotStart, plotEnd)
    plt.ylim(-400,400)
    plt.xlabel("Time (seconds)")
    plt.ylabel("Force (nt)")
    plt.legend()
    plt.subplot(2,2,3)
    plt.plot(times, residuals)
    if passFail[primaryBump] == 6:
        plt.text(2.0, 75.0, f"RMS = {rmsError:.2f}", color='g')
    elif passFail[primaryBump] == 7:
        plt.text(2.0, 75.0, f"RMS = {rmsError:.2f}", color='r')
    plt.xlim(plotStart, plotEnd)
    plt.ylim(-100,100)
    plt.xlabel("Time (seconds)")
    plt.ylabel("Residuals (nt)")
    
    if secondaryName is not None:
        plotStart = thisBump[thisBump[secondaryBump]==2]['timestamp'].values[0] - 1.0
        plotEnd = plotStart + 14.0
        start = Time(plotStart, format='unix_tai', scale='tai')
        end = Time(plotEnd, format='unix_tai', scale='tai')
        forces = await client.select_time_series("lsst.sal.MTM1M3.forceActuatorData", [secondaryForce, 'timestamp'], start.utc, end.utc)
        times = forces['timestamp'].values
        t0 = times[0]
        times -= t0
        secondaryForces = forces[secondaryForce].values
        residuals = secondaryForces-averageSecondarySpline(times)
        rmsError = np.sqrt(np.mean(residuals**2))
        plotStart -= t0
        plotEnd -= t0
        plt.subplot(2,2,2)
        plt.title(f"Secondary - {secondaryName}")
        plt.plot(times, averageSecondarySpline(times), label='Average')
        plt.plot(times, secondaryForces, label='Data')
        if passFail[primaryBump] == 6:
            plt.text(2.0, 350.0, "PASSED", color='g')
        elif passFail[primaryBump] == 7:
            plt.text(2.0, 350.0, "FAILED", color='r')
        plt.xlim(plotStart, plotEnd)
        plt.ylim(-400,400)
        plt.xlabel("Time (seconds)")
        plt.ylabel("Force (nt)")
        plt.legend()
        plt.subplot(2,2,4)
        plt.plot(times, residuals)
        if passFail[primaryBump] == 6:
            plt.text(2.0, 75.0, f"RMS = {rmsError:.2f}", color='g')
        elif passFail[primaryBump] == 7:
            plt.text(2.0, 75.0, f"RMS = {rmsError:.2f}", color='r')
        plt.xlim(plotStart, plotEnd)
        plt.ylim(-100,100)
        plt.xlabel("Time (seconds)")
        plt.ylabel("Residuals (nt)")
    else:
        plt.subplot(2,2,2)
        plt.title("No Secondary")
        plt.xticks([])
        plt.yticks([])
        plt.subplot(2,2,4)
        plt.xticks([])
        plt.yticks([])
    return

## This cell will make the residual plots for a single actuator

In [ ]:
bumps = await client.select_time_series("lsst.sal.MTM1M3.logevent_forceActuatorBumpTestStatus", "*", \
                                        Time(residualStart, scale='utc'), Time(residualEnd, scale='utc'))
timestamp = bumps.index[0].isoformat().split('.')[0].replace('-','').replace(':','')
id = 324
fig = plt.figure(figsize=(10,10))
await plotBumpResultsAndResiduals(fig, bumps, averageSplineDict, id)
plt.savefig(dataDir + f"Bump_Test_Residuals_{id}_{timestamp}.png")

## This will make plots of the residuals for the whole bump test

In [ ]:
bumps = await client.select_time_series("lsst.sal.MTM1M3.logevent_forceActuatorBumpTestStatus", "*", \
                                        Time(residualStart, scale='utc'), Time(residualEnd, scale='utc'))
timestamp = bumps.index[0].isoformat().split('.')[0].replace('-','').replace(':','')
pdf = PdfPages(f"/home/c/cslage/u/MTM1M3/data/Bump_Test_Residuals_{timestamp}.pdf")

for index in range(len(FATABLE)):
    id = FATABLE[index][M1M3FATable.FATABLE_ID]
    fig = plt.figure(figsize=(10,10))
    await plotBumpResultsAndResiduals(fig, bumps, averageSplineDict, id)
    pdf.savefig(fig)  # saves the current figure into a pdf page
    plt.close()
pdf.close()
